In [98]:
import pandas as pd
import numpy as np
import datetime
import time
from tqdm import tqdm

train_data = pd.read_csv('features.csv',index_col='match_id')
test_data = pd.read_csv('features_test.csv',index_col='match_id')

In [99]:
columns_with_nan = train_data.columns[train_data.isna().any()].tolist()
columns_with_nan, len(columns_with_nan)

(['first_blood_time',
  'first_blood_team',
  'first_blood_player1',
  'first_blood_player2',
  'radiant_bottle_time',
  'radiant_courier_time',
  'radiant_flying_courier_time',
  'radiant_first_ward_time',
  'dire_bottle_time',
  'dire_courier_time',
  'dire_flying_courier_time',
  'dire_first_ward_time'],
 12)

In [100]:
for column in columns_with_nan:
    train_data[column].fillna(0, inplace=True)
    test_data[column].fillna(0, inplace=True)
    
x_train = np.array(train_data.values[:,1:-6])
y_train = np.array(train_data.values[:,-5])

In [4]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold, cross_val_score

num_trees = [5,10,20,30,40,50]
results = {}

splitter = KFold(n_splits=5, random_state=241, shuffle=True)
for num in num_trees:
    clf = GradientBoostingClassifier(n_estimators=num,learning_rate=0.1,verbose=False,random_state=241)
    # clf.fit(x_train, y_train)
    result = cross_val_score(estimator=clf, X=x_train, y=y_train, scoring='roc_auc' ,cv=splitter,n_jobs=-1)
    results[num] =  result
results

{5: array([0.64010969, 0.62849414, 0.63656996, 0.63369292, 0.63982151]),
 10: array([0.66943496, 0.65627754, 0.66390454, 0.66281223, 0.66950933]),
 20: array([0.68939011, 0.67567277, 0.68201026, 0.67977584, 0.68741889]),
 30: array([0.69641668, 0.68365441, 0.68731862, 0.68728458, 0.69280674]),
 40: array([0.70087493, 0.68889686, 0.69145856, 0.69219074, 0.69727685]),
 50: array([0.70374327, 0.69328481, 0.69471307, 0.69565719, 0.7000093 ])}

In [5]:
import time
import datetime

start_time = datetime.datetime.now()

clf = GradientBoostingClassifier(n_estimators=30,learning_rate=0.1,verbose=False,random_state=241)
result = cross_val_score(estimator=clf, X=x_train, y=y_train, scoring='roc_auc' ,cv=splitter,n_jobs=-1)

print('Time elapsed:', datetime.datetime.now() - start_time)

Time elapsed: 0:01:02.719202


In [7]:
sum(x for x in results[30]) / 5., sum(x for x in results[50]) / 5. 

(0.6894962060591201, 0.6974815295110693)

Второй этап выполнения проекта

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_x_train = scaler.fit_transform(x_train)

__C__ = np.power(10.0, np.arange(-5, 6))
results = {}
for c in tqdm(__C__):
    tic = datetime.datetime.now()
    clf = LogisticRegression(penalty='l2', C=c, n_jobs=-1)
    results[c] = cross_val_score(estimator=clf, X=scaled_x_train, y=y_train, scoring='roc_auc', cv=splitter, n_jobs=-1)
    toc = datetime.datetime.now()
    print(c,'  ',toc-tic)
results


  0%|          | 0/11 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

  9%|▉         | 1/11 [00:01<00:19,  1.99s/it]

1e-05    0:00:01.986121


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

 18%|█▊        | 2/11 [00:04<00:22,  2.45s/it]

0.0001    0:00:02.907532


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

 27%|██▋       | 3/11 [00:09<00:25,  3.20s/it]

...

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

 91%|█████████ | 10/11 [00:55<00:05,  5.53s/it]

10000.0    0:00:06.504719


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

100%|██████████| 11/11 [01:01<00:00,  5.64s/it]


100000.0    0:00:06.722953


{1e-05: array([0.69951301, 0.6915458 , 0.69425442, 0.69426995, 0.69585829]),
 0.0001: array([0.71593961, 0.70822324, 0.70961019, 0.70997155, 0.71195314]),
 0.001: array([0.72104209, 0.71352306, 0.71421763, 0.71481709, 0.71682199]),
 0.01: array([0.72130764, 0.71360887, 0.71432198, 0.71507859, 0.71697901]),
 0.1: array([0.72130087, 0.71354254, 0.71429145, 0.71506754, 0.71696029]),
 1.0: array([0.72129927, 0.71353427, 0.71428785, 0.71506292, 0.71695754]),
 10.0: array([0.72129884, 0.71353396, 0.71428692, 0.71506335, 0.71695754]),
 100.0: array([0.72129896, 0.71353404, 0.71428691, 0.71506351, 0.7169572 ]),
 1000.0: array([0.721299  , 0.71353413, 0.71428689, 0.71506352, 0.71695717]),
 10000.0: array([0.721299  , 0.71353412, 0.71428689, 0.7150635 , 0.71695715]),
 100000.0: array([0.721299  , 0.71353412, 0.71428689, 0.71506351, 0.71695715])}

In [25]:
best_c = 1e-5
best_score = 0
for c in results:
    score = max(results[c])
    if score > best_score:
        best_c, best_score = c, score
best_c, best_score

(0.01, 0.72130764244798)

In [82]:
reduce_data = train_data.drop(columns=['lobby_type','r1_hero','r2_hero','r3_hero','r4_hero','r5_hero',
                                                    'd1_hero','d2_hero','d3_hero','d4_hero','d5_hero'])
x_train = np.array(reduce_data.values[:,1:-6])
y_train = np.array(reduce_data.values[:,-5])

In [27]:
results = {}
scaled_x_train = scaler.fit_transform(x_train)

for c in tqdm(__C__):
    tic = datetime.datetime.now()
    clf = LogisticRegression(penalty='l2', C=c, n_jobs=-1)
    results[c] = cross_val_score(estimator=clf, X=scaled_x_train, y=y_train, scoring='roc_auc', cv=splitter, n_jobs=-1)
    toc = datetime.datetime.now()
    print(c,'  ',toc-tic)
results


  0%|          | 0/11 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

  9%|▉         | 1/11 [00:01<00:17,  1.79s/it]

1e-05    0:00:01.790874


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

 18%|█▊        | 2/11 [00:04<00:19,  2.15s/it]

0.0001    0:00:02.498976


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

 27%|██▋       | 3/11 [00:08<00:22,  2.80s/it]

...

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

 91%|█████████ | 10/11 [00:49<00:04,  4.99s/it]

10000.0    0:00:06.203665


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

100%|██████████| 11/11 [00:55<00:00,  5.09s/it]


100000.0    0:00:06.028103


{1e-05: array([0.69953085, 0.69144709, 0.69426417, 0.69418899, 0.69567937]),
 0.0001: array([0.71606209, 0.70813587, 0.7097538 , 0.70998746, 0.7117414 ]),
 0.001: array([0.72119316, 0.71346423, 0.71447732, 0.71488578, 0.71667335]),
 0.01: array([0.72146903, 0.71352779, 0.71460349, 0.71516705, 0.71683384]),
 0.1: array([0.72146005, 0.71345427, 0.71458578, 0.71516408, 0.71681749]),
 1.0: array([0.72145576, 0.71344656, 0.71458279, 0.71516256, 0.7168144 ]),
 10.0: array([0.72145495, 0.7134459 , 0.7145824 , 0.71516187, 0.71681328]),
 100.0: array([0.7214552 , 0.71344606, 0.71458229, 0.7151619 , 0.71681347]),
 1000.0: array([0.72145513, 0.71344607, 0.71458231, 0.71516185, 0.71681336]),
 10000.0: array([0.72145512, 0.71344605, 0.71458233, 0.71516185, 0.71681336]),
 100000.0: array([0.72145512, 0.71344605, 0.71458233, 0.71516186, 0.71681338])}

In [28]:
best_c = 1e-5
best_score = 0
for c in results:
    score = max(results[c])
    if score > best_score:
        best_c, best_score = c, score
best_c, best_score

(0.01, 0.7214690299501316)

In [29]:
from collections import Counter
for i in range(10):
    column = ''
    if i < 5:
        column = 'r'+str(i+1)+'_hero'
    else:
        column = 'd'+str(i-4)+'_hero'
    print(len(Counter(train_data[column]).keys()))
# Всего 108 различных героев

108
108
108
108
108
108
108
108
108
108


In [81]:
X_pick = np.zeros((train_data.shape[0], 112))

for i, match_id in enumerate(train_data.index):
    for p in range(5):
        X_pick[i, train_data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, train_data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [86]:
x_train = np.concatenate((x_train, X_pick), axis=1)

In [88]:
results = {}
scaled_x_train = scaler.fit_transform(x_train)

for c in tqdm(__C__):
    tic = datetime.datetime.now()
    clf = LogisticRegression(penalty='l2', C=c, n_jobs=-1)
    results[c] = cross_val_score(estimator=clf, X=scaled_x_train, y=y_train, scoring='roc_auc', cv=splitter, n_jobs=-1)
    toc = datetime.datetime.now()
    print(c,'  ',toc-tic)
results


  0%|          | 0/11 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

  9%|▉         | 1/11 [00:02<00:28,  2.83s/it]

1e-05    0:00:02.824150


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

 18%|█▊        | 2/11 [00:07<00:32,  3.63s/it]

0.0001    0:00:04.435635


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

 27%|██▋       | 3/11 [00:15<00:40,  5.04s/it]

...

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

 91%|█████████ | 10/11 [01:34<00:09,  9.47s/it]

10000.0    0:00:11.532054


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))

100%|██████████| 11/11 [01:46<00:00,  9.64s/it]


100000.0    0:00:11.332690


{1e-05: array([0.71899764, 0.71111706, 0.71517842, 0.71282182, 0.71577733]),
 0.0001: array([0.74660913, 0.73994937, 0.74275384, 0.74006535, 0.74413576]),
 0.001: array([0.75480929, 0.74921748, 0.75180115, 0.74875271, 0.75304767]),
 0.01: array([0.75486459, 0.74975739, 0.75213434, 0.74921956, 0.75316565]),
 0.1: array([0.7547857 , 0.74978582, 0.75207819, 0.74924222, 0.75303333]),
 1.0: array([0.75477379, 0.74978743, 0.75206926, 0.74924338, 0.75300847]),
 10.0: array([0.75477304, 0.74978721, 0.75206819, 0.7492435 , 0.75300589]),
 100.0: array([0.75477292, 0.74978686, 0.75206834, 0.74924341, 0.75300582]),
 1000.0: array([0.75477295, 0.74978682, 0.75206821, 0.74924337, 0.7530057 ]),
 10000.0: array([0.75477296, 0.74978682, 0.75206821, 0.74924335, 0.75300573]),
 100000.0: array([0.75477296, 0.74978682, 0.75206822, 0.74924336, 0.75300574])}

In [89]:
best_c = 1e-5
best_score = 0
for c in results:
    score = max(results[c])
    if score > best_score:
        best_c, best_score = c, score
best_c, best_score

(0.01, 0.7548645941913376)

In [90]:
best_clf = LogisticRegression(penalty='l2', C=0.01, n_jobs=-1)
best_clf.fit(scaled_x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [96]:
test_data = test_data.drop(columns=['lobby_type','r1_hero','r2_hero','r3_hero','r4_hero','r5_hero',
                                                    'd1_hero','d2_hero','d3_hero','d4_hero','d5_hero'])
x_test = np.array(test_data.values[:,1:])

In [101]:
X_pick = np.zeros((test_data.shape[0], 112))

for i, match_id in enumerate(test_data.index):
    for p in range(5):
        X_pick[i, test_data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, test_data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
x_test = np.concatenate((x_test, X_pick), axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [104]:
pred = best_clf.predict_proba(x_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [109]:
pred[200]

array([1.0000000e+000, 4.8411466e-184])